# Data Analysis with ACADEMY methodology

Here is an example of what it is possible to do using the ACADEMY methodology. 
For more informations please read Zanetta et al,. 2018 
Don't hesitate to contact the author via Github or directly by email.

## Author 

19/09/2018 Pierre-marie Zanetta

## requirements

HyperSpy 1.3 and need to merge the Pull request #1835 and #1462

We first import the different pacakges necessary to run the whole code

In [1]:
%matplotlib qt 
#nbagg
import numpy as np
import matplotlib.pyplot as plt
#import hyperspy.api as hs 

To obtain a merged version that run the entire code it is possible to download the branch "master_merged" on my personnal github repository:
https://github.com/ZanettaPM/hyperspy/tree/Master_merged

clone this repository on a github folder in your documents and remplace the path below

In [2]:
import imp
hs=imp.load_source("hyperspy", "D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy/__init__.py")
hs=imp.load_source("hyperspy.api", "D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy/api.py")

In [3]:
hs.hyperspy.Release.version

'1.4.dev'

## 1. Description of the sample and getting started

The sample consists of a cropped area of the map "Zone 2" of the meteorite of Paris presented in the paper.
This sample is investigated using a FEG-SEM JEOL JSM-7800F LV at the University of Lille equipped with an EDX/EBSD Aztec system from Oxford Instruments and a silicon drift detector (SDD XMaxN) of 80 mm². 

Initially the raw file extracted from Aztech was composed of 3 files: ".msa", ".rpl" and the ".raw"
The ".msa" file contains a single pixel spectrum and the metadata to calibrate it. The ".rpl" is a header to read the ".raw". The ".raw" file contains an EDS/SEM spectral image.
These data have been cropped and converted into the hyperspy format ".hspy", and the metadata have been completed

In [4]:
s=hs.load("data.hspy")
s

<EDSSEMSpectrum, title: Map Sum Spectrum, dimensions: (97, 110|512)>

In [5]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 29.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 170393.59375
│       │       └── real_time = 210312.96875
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── beam_energy = 5.0
│       └── magnification = X440
├── General
│   ├── date = 2017-01-24
│   ├── original_filename = Paris_section-6_Map2
│   ├── time = 17:42:00
│   └── title = Map Sum Spectrum
├── Sample
│   ├── elements = ['Al', 'C', 'Ca', 'Fe', 'Mg', 'Na', 'Ni', 'O', 'S', 'Si']
│   └── xray_lines = ['Al_Ka', 'C_Ka', 'Ca_La', 'Fe_La', 'Mg_Ka', 'Na_Ka', 'Ni_La', 'O_Ka', 'S_Ka', 'Si_Ka']
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_SEM

We plot the hyperspectral signal and the different X-ray lines.
The signal have been calibrated in energy beforehand

In [6]:
s.plot(True) 

## 2. Creating a model with gaussians and the new background formulation

To obtain accurate elemental maps, we create a model to remove the background and we fit gaussian functions to the hyperspectral map.

The default background proposed by hyperspy is a polynome function. Here we don't use this components but we add another background taking into account the emission and the absorption within the sample and the detector. 
To see the code behind this component please refer to the pull request #1835

In [7]:
m=s.create_model(auto_background=False)

The thin window in front of our detector is a polymer window. We use the efficiency curve modelised beforehand.

In [8]:
m.add_physical_background(detector='Polymer_C')

In first instance we approximate the composition of each pixel based on the height of the peaks. This approximation is necessary to obtain a shape of of the background which fit well with the data.

In [9]:
m.components.Bremsstrahlung.initialize()

D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

{'Quant map has been created'}

We print the differents components of the model and we can delete X-ray lines with weak intensity to speed up the fitting procedure.

In [10]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Ca_Ll |               Ca_Ll |            Gaussian
   7 |               Ca_Ln |               Ca_Ln |            Gaussian
   8 |               Fe_La |               Fe_La |            Gaussian
   9 |               Fe_Ll |               Fe_Ll |            Gaussian
  10 |               Fe_Ln |               Fe_Ln |            Gaussian
  11 |              Fe_Lb3 |              Fe_Lb3 |            Gaussian
  12 |

In [11]:
m.remove([1,6,7,9,10,11,13,15,20])

In [12]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |                C_Ka |                C_Ka |            Gaussian
   2 |               Ca_Ka |               Ca_Ka |            Gaussian
   3 |               Ca_Kb |               Ca_Kb |            Gaussian
   4 |               Ca_La |               Ca_La |            Gaussian
   5 |               Fe_La |               Fe_La |            Gaussian
   6 |               Mg_Ka |               Mg_Ka |            Gaussian
   7 |               Na_Ka |               Na_Ka |            Gaussian
   8 |               Ni_La |               Ni_La |            Gaussian
   9 |               Ni_Ll |               Ni_Ll |            Gaussian
  10 |               Ni_Ln |               Ni_Ln |            Gaussian
  11 |              Ni_Lb1 |              Ni_Lb1 |            Gaussian
  12 |

Finally, we fit the background to the data. The background cannot be negative, in consequence we fix bounds. 

In [13]:
m.fit_background(kind='multi',bounded=True, windows_sigma=([3,3]))

D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

And we fit the gaussians using the linear fit proposed by thomas aarholt (already merged in "master_merged")
https://github.com/hyperspy/hyperspy/pull/1462

In [15]:
m.fix_background()

In [16]:
m.multifit(fitter='linear',bounded=True)

(512,)


D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

(512,)
(512,)
(512,)


(512,)
(512,)
(512,)
(512,)
(512,)
(512,)


(512,)
(512,)
(512,)
(512,)


...

(512,)
(512,)
(512,)
(512,)


(512,)
(512,)
(512,)
(512,)


(512,)
(512,)


(512,)
(512,)


(512,)
(512,)
(512,)
(512,)



In [54]:
m.plot(plot_components=True)

D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

In [59]:
elemap=m.get_lines_intensity()
plt.figure()
plt.imshow(elemap[5],cmap='jet')
plt.title('Mg map, background corrected, in number of counts')
plt.colorbar()

## 4. mineral map and MLLS procedure

The phase map have been obtained using XmapTools software and manual refinement on matlab software. However, such clustering methods exist in python and have been proposed into hyperspy: 
https://github.com/hyperspy/hyperspy/pull/1353

Here to continue the example, a file containing the value of the map is provided in the repository of this tutorial

In [223]:
carto=hs.load('cartography.hspy')

In [257]:
phases=['Metal Grain','Olivine','Pyroxene','Fe-rich sulfides','Epoxy','Pentlandite','Mg-rich phyll/amorph','Fe-rich phyll/amorph','Carbonates'
,'Spinel','sulfates','Olivine (Fo<90)']

In [258]:
plt.imshow(carto,plt.get_cmap('jet', 11))
cb = plt.colorbar()    
cb.set_ticks([1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5])
cb.set_ticklabels(phases)


The modelised background is converted into a signal.(enfin ça c'est si cela marche)

bgd=m.as_signal(component_list=[3])

(s+bgd*1j).plot()

Then we remove the background from the hyperspectral data

s2=s-bgd
mask=s2>0
s2=s2*mask

s2.plot(True)

We create a new model that will contain the refence spectra

In [21]:
s2=s

In [22]:
m2 = s2.create_model(False,False)

ideally we use the spectrum from reference grains. Here a simple mean is used for easier treatments 

In [224]:
carto=np.array(carto.data)
a=int(np.max(carto))
Sref=dict()
indice=dict()
for i in range (1,a+1):
    s2=s.data[carto==i]
    s2=hs.signals.EDSSEMSpectrum(s2)
    s2=s2.mean()
    s2.get_calibration_from(s) 
    s2.metadata=s.metadata 
    Sref[i]=s2
    indice[i]=i

In [24]:
plt.figure()
plt.plot(Sref[8])

In [25]:
for i in range (1,a+1):
    g=hs.model.components1D.ScalableFixedPattern(Sref[i])
    m2.extend([g])

In [26]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   1 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   2 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   3 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   4 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   5 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   6 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   7 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   8 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   9 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  10 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  11 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..

In [27]:
for i in range (0,len(m2)):
    m2.set_parameters_not_free([i], parameter_name_list=['xscale','shift'])
    m2.set_parameters_value('yscale',value=0.5,component_list=[i])
    m2[i].yscale.bmin=0
    m2[i].yscale.bmax=1

In [28]:
m2.multifit(fitter='linear',bounded=True, grad=True)

In [29]:
m2.plot(plot_components=True)

In [115]:
for i in range(0,len(m2)):
    a=m2[i].yscale.map['values']
    plt.figure()
    plt.imshow(a,cmap='jet')
    plt.colorbar()

# 5. Density map

We have now corrected our modal abundances and the different reference grains have been analyzed using EPMA. Thanks to this measurements we know the composition of the different phases. Thoses compositions will be used for the density measurement of the hyperspectral map. 

In [31]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 29.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 170393.59375
│       │       └── real_time = 210312.96875
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── beam_energy = 5.0
│       └── magnification = X440
├── General
│   ├── date = 2017-01-24
│   ├── original_filename = Paris_section-6_Map2
│   ├── time = 17:42:00
│   └── title = Map Sum Spectrum
├── Sample
│   ├── elements = ['Al', 'C', 'Ca', 'Fe', 'Mg', 'Na', 'Ni', 'O', 'S', 'Si']
│   └── xray_lines = ['Al_Ka', 'C_Ka', 'Ca_La', 'Fe_La', 'Mg_Ka', 'Na_Ka', 'Ni_La', 'O_Ka', 'S_Ka', 'Si_Ka']
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_SEM

In [32]:
s.metadata.Sample.elements=[]
s.metadata.Sample.xray_lines=[]
s.metadata.Sample.elements=['C','Na','O','Si','Fe','Al','Mg','S','Ni','Ca']
s.add_lines()

In [33]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 29.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 170393.59375
│       │       └── real_time = 210312.96875
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── beam_energy = 5.0
│       └── magnification = X440
├── General
│   ├── date = 2017-01-24
│   ├── original_filename = Paris_section-6_Map2
│   ├── time = 17:42:00
│   └── title = Map Sum Spectrum
├── Sample
│   ├── elements = ['Al', 'C', 'Ca', 'Fe', 'Mg', 'Na', 'Ni', 'O', 'S', 'Si']
│   └── xray_lines = ['Al_Ka', 'C_Ka', 'Ca_La', 'Fe_La', 'Mg_Ka', 'Na_Ka', 'Ni_La', 'O_Ka', 'S_Ka', 'Si_Ka']
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_SEM

In [154]:
Metal_grain=[0,1,0,95,0,0,4,0,0,0]
Olivine=[0,1,0,1.7,30,0,0,43.28,0,18.7]
Pyroxene=[0,1,0,0,22.9,0,0,48.32,0,28.54]
Fe_sulfides=[0,1,0,55.94,0,0,3.80,0,34.6,0]
Epoxy=[0,70,0,0,0,0,0,30,0,0]
Pentlandite=[0,1,0,32.5,0,0,22.5,0,27.3,0]
Mg_phyll=[1.599,1,0.87,16.05,10.20,0.6,1.2,33.27,3.30,14.059]
Fe_phyll=[1.67,1,0.39,20.87,7.43,0.7,1.17,29.75,2.22,12.5572]
Carbonates=[0,40,39.46,1.11,0,0.3,0,17.31,0,0]
Spinel=[0,0,3.64,19.751,8,0,0,36.89,3.6,12.83]
Sulfate=[2,13.02,8.50,12.34,7.49,2,0,33.52,6,11]
Fayalite=[0,0,0,16,23.21,0,0,41.6,0,18.6]
Elmt=[Metal_grain,Olivine,Pyroxene,Fe_sulfides,Epoxy,Pentlandite,Mg_phyll,Fe_phyll,Carbonates,Spinel,Sulfate,Fayalite]

In [155]:
x=np.shape(carto)

In [156]:
mask2=[]
mask2=np.ones([x[0],x[1],10])
mask2=hs.signals.EDSSEMSpectrum(mask2)

In [157]:
for i in range (0,12):
    a=np.where(carto==(i+1))
    x=np.shape(a)
    for u in range (0,x[1]):
        g=a[0]
        h=a[1]
        mask2.inav[h[u],g[u]] =Elmt[i]
mask2.plot()

In [38]:
quanti=np.array(mask2.data,float)
np.shape(quanti)

(110, 97, 10)

In [39]:
m3=s.create_model(auto_background=False)

In [40]:
m3.add_physical_background(detector='Polymer_C',quantification=quanti)

In [41]:
m3.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Ca_Ll |               Ca_Ll |            Gaussian
   7 |               Ca_Ln |               Ca_Ln |            Gaussian
   8 |               Fe_La |               Fe_La |            Gaussian
   9 |               Fe_Ll |               Fe_Ll |            Gaussian
  10 |               Fe_Ln |               Fe_Ln |            Gaussian
  11 |              Fe_Lb3 |              Fe_Lb3 |            Gaussian
  12 |

In [42]:
m3.remove([1,6,7,9,10,11,13,15,20])

In [43]:
m3.components.Bremsstrahlung.initialize()

D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

{'Quant map has been created'}

In [44]:
m3.fit_background(kind='multi',bounded=True, windows_sigma=[3,3])

D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

In [46]:
m3.plot(plot_components=True)

D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:307: RuntimeWarning: divide by zero encountered in log
  mac_res = np.exp(np.log(macs[index - 1]) +
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:308: RuntimeWarning: divide by zero encountered in true_divide
  np.log(macs[index] / macs[index - 1]) *
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: divide by zero encountered in true_divide
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:309: RuntimeWarning: invalid value encountered in log
  (np.log(energies / energies_db[index - 1]) /
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: divide by zero encountered in true_divide
  np.log(energies_db[index] / energies_db[index - 1])))
D://Users/Documents/GitHub/ZanettaPM/hyperspy/hyperspy\misc\material.py:310: RuntimeWarning: invalid valu

In [47]:
coef=(m3.components.Bremsstrahlung.coefficients.map['values'][:][:][:])

In [48]:
plt.figure()
plt.imshow(coef[:,:,0],cmap='jet')
plt.colorbar()

In [49]:
Density=[8.1,3.27,3.4,4.61,1.2,5,2.88,2.9,1.6,3.72,2.75,4]
x=np.shape(carto)
Dnom=np.ones([x[0],x[1],1])
Dnom=hs.signals.Signal1D(Dnom)


for i in range (1,np.max(carto)):
    a=np.where(carto==(i))
    x=np.shape(a)
    for u in range (0,x[1]):
        g=a[0]
        h=a[1]
        Dnom.inav[h[u],g[u]] =Density[i-1]

In [50]:
mask=coef[:,:,0]>8
coef[:,:,0][mask]=np.nan        
mask=Dnom.data[:,:,0]<1.2
Dnom.data[:,:,0][mask]=np.nan
Dnom.plot()

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
C:\Anaconda3\lib\site-packages\matplotlib\axes\_base.py:3152: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0.0, right=0.0
  'left=%s, right=%s') % (left, right))


In [51]:
Density=np.ones(np.max(carto))
Proxy=np.ones(np.max(carto))
for i in range (1,np.max(carto)):
    mask=carto==i
    Density[i]=np.nanmean(Dnom.data[:,:,0][mask])
    Proxy[i]=np.nanmean(coef[:,:,0][mask])

In [52]:
plt.figure()
plt.plot(Density,Proxy,'.')

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [70]:
from scipy import stats
x=np.transpose(Density)
y=Proxy
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
plt.plot(x, y, '.', label='data')
plt.plot(x, intercept + slope*x, 'r', label='fit')

#plt.title('Density proxy compare to real density')
plt.xlabel('Real Density (g/cm3)')
plt.ylabel('Fitted Proxy (Rho.x)')

Text(0,0.5,'Fitted Proxy (Rho.x)')

In [123]:
a=slope
b=intercept
Dcalc=((coef[:,:,0]-intercept)/slope)
mask=Dcalc>10
Dcalc[mask]=np.nan
plt.imshow(Dcalc,cmap='jet')
plt.colorbar()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


Ecrire script ou tu multiplie quantmap*density*abonndance 

On extrait les AM corrigées par MLLS

In [150]:
AMfinal=np.ones(np.max(carto)-1)
for i in range (1,np.max(carto)):
    a=np.sum(m2[i].yscale.map['values']);
    AMfinal[i-1]=a/(s.axes_manager[0].size*s.axes_manager[1].size)

In [151]:
AMfinal

array([ 0.06229636,  0.28616776,  0.02943773,  0.00085856,  0.01319227,
        0.28604487,  0.17556515,  0.0065105 ,  0.00693344,  0.08398093,
        0.0078344 ])

On extrait les densité moyennes pour chaque phases

In [152]:
Dfinal=np.ones(np.max(carto)-1)
for i in range (1,np.max(carto)):
    maskD=Dcalc[carto==i]
    Dfinal[i-1]=np.nanmean(maskD)

In [153]:
Dfinal

array([ 9.32725553,  3.93548878,  3.65189789,  3.28550977,  1.72430852,
        2.94007917,  3.00369902,  2.8963461 ,  1.32030649,  2.67127902,
        2.0601809 ])

In [271]:
print('|-----------------------------|')
print('|     Weight compositions     |')
print('|-----------------------------|')

print('| Phases       | Modal Abundances in %  |   Density in g/cm3    |')
print('|--------------|------------------------|-----------------------|')
print('| Al (Wt. %)   |         {:.2f}           |         {:.2f}          |'.format(AM[0],Dfinal[0]))
print('| Al (Wt. %)   |         {:.2f}           |         {:.2f}          |'.format(AM[1],Dfinal[1]))
print('| Al (Wt. %)   |         {:.2f}           |         {:.2f}          |'.format(AM[2],Dfinal[2]))
print('| Al (Wt. %)   |         {:.2f}           |         {:.2f}          |'.format(AM[3],Dfinal[3]))
print('| Al (Wt. %)   |         {:.2f}           |         {:.2f}          |'.format(AM[4],Dfinal[4]))
print('| Al (Wt. %)   |         {:.2f}           |         {:.2f}          |'.format(AM[5],Dfinal[5]))
print('|--------------|------------------------|-----------------------|')

|-----------------------------|
|     Weight compositions     |
|-----------------------------|
| Phases       | Modal Abundances in %  |   Density in g/cm3    |
|--------------|------------------------|-----------------------|
| Al (Wt. %)   |         0.06           |         9.33          |
| Al (Wt. %)   |         0.29           |         3.94          |
| Al (Wt. %)   |         0.03           |         3.65          |
| Al (Wt. %)   |         0.00           |         3.29          |
| Al (Wt. %)   |         0.01           |         1.72          |
| Al (Wt. %)   |         0.29           |         2.94          |
|--------------|------------------------|-----------------------|


In [227]:
BulkQuant=np.empty([len(s.metadata.Sample.elements)])
for i in range (0,np.max(carto)-1):        
    for k in range (0,len(s.metadata.Sample.elements)):
        BulkQuant[k]+=(AMfinal[i]*Dfinal[i]/np.mean(Dfinal))*Elmt[i][k]

In [268]:
print('|-----------------------------|')
print('|     Weight compositions     |')
print('|-----------------------------|')

print('| Elements     |   Qti in Wt% |')
print('|--------------|--------------|')
print('| Al (Wt. %)   |     {:.2f}     |'.format(BulkQuant[0]))
print('| C  (Wt. %)   |     {:.2f}     |'.format(BulkQuant[1]))
print('| Ca (Wt. %)   |     {:.2f}     |'.format(BulkQuant[2]))
print('| Fe (Wt. %)   |     {:.2f}    |'.format(BulkQuant[3]))
print('| Mg (Wt. %)   |     {:.2f}    |'.format(BulkQuant[4]))
print('| Na (Wt. %)   |     {:.2f}     |'.format(BulkQuant[5]))
print('| Ni (Wt. %)   |     {:.2f}     |'.format(BulkQuant[6]))
print('| O  (Wt. %)   |     {:.2f}    |'.format(BulkQuant[7]))
print('| S  (Wt. %)   |     {:.2f}    |'.format(BulkQuant[8]))
print('| Si (Wt. %)   |     {:.2f}    |'.format(BulkQuant[9]))
print('|--------------|--------------|')

|-----------------------------|
|     Weight compositions     |
|-----------------------------|
| Elements     |   Qti in Wt% |
|--------------|--------------|
| Al (Wt. %)   |     0.37     |
| C  (Wt. %)   |     1.98     |
| Ca (Wt. %)   |     0.78     |
| Fe (Wt. %)   |     46.30    |
| Mg (Wt. %)   |     14.05    |
| Na (Wt. %)   |     2.11     |
| Ni (Wt. %)   |     9.74     |
| O  (Wt. %)   |     29.41    |
| S  (Wt. %)   |     17.69    |
| Si (Wt. %)   |     30.41    |
|--------------|--------------|


In [239]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 29.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 170393.59375
│       │       └── real_time = 210312.96875
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── beam_energy = 5.0
│       └── magnification = X440
├── General
│   ├── date = 2017-01-24
│   ├── original_filename = Paris_section-6_Map2
│   ├── time = 17:42:00
│   └── title = Map Sum Spectrum
├── Sample
│   ├── elements = ['Al', 'C', 'Ca', 'Fe', 'Mg', 'Na', 'Ni', 'O', 'S', 'Si']
│   └── xray_lines = ['Al_Ka', 'C_Ka', 'Ca_La', 'Fe_La', 'Mg_Ka', 'Na_Ka', 'Ni_La', 'O_Ka', 'S_Ka', 'Si_Ka']
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_SEM

In [185]:
Dfinal[10]

2.0601809044596995

In [243]:
BulkQuant=np.empty([len(s.metadata.Sample.elements)])
for i in range (0,1):        
    for k in range (0,len(s.metadata.Sample.elements)):
        BulkQuant[k]+=(AMfinal[i]*Dfinal[i]/np.mean(Dfinal))*Elmt[i][k]

In [244]:
BulkQuant

array([  0.37099243,   1.97908224,   0.78217215,  46.30378942,
        14.05154105,   2.10894529,   9.74136018,  29.41046158,
        17.69159169,  30.40794821])

In [ ]:
whos